In [1]:
import os
import struct
import tkinter as tk
from tkinter import filedialog
import glob
import pandas as pd

In [7]:
import os
import glob
import tkinter as tk
from tkinter import filedialog
import struct
import pandas as pd

def select_folder():
    root = tk.Tk()
    root.withdraw()  # GUIを隠す
    folder_path = filedialog.askdirectory()
    return folder_path

def find_files(folder_path, filename):
    # サブフォルダを含めて指定のファイルを検索
    return glob.glob(os.path.join(folder_path, '**', filename), recursive=True)

def read_ect_file(file_path):
    with open(file_path, 'rb') as file:
        data = file.read()
    
    # 2バイトずつ読み込み、リトルエンディアンとして解釈して整数を取得
    num_integers = len(data) // 2
    integers = [struct.unpack('<H', data[i*2:i*2+2])[0] for i in range(num_integers)]
    return integers

def process_judge_2_ect(integers):
    results = []
    max_length = 0  # 最大長を保持する変数

    while len(integers) > 0:
        if len(integers) < 4:
            break
        
        if integers[3] == 0:
            # 初めから4番目が0だった場合、7番目までを切り取る
            if len(integers) >= 7:
                result = integers[:7]
                max_length = max(max_length, len(result))  # 最大長を更新
                results.append(result)
                integers = integers[7:]
            else:
                break
        else:
            # 初めから4番目が0でない場合、(4番目の数字×12)+7までを切り取る
            cut_index = (integers[3] * 12) + 7
            if len(integers) >= cut_index:
                result = integers[:cut_index]
                max_length = max(max_length, len(result))  # 最大長を更新
                results.append(result)
                integers = integers[cut_index:]
            else:
                break

    # 0埋めを行う
    for result in results:
        result += [0] * (max_length - len(result))  # 最大長に満たない要素を0で埋める
    
    return results

def write_to_excel(results, folder_name):
    # 新しいExcelファイルを作成
    excel_file_name = f"{folder_name}.xlsx"
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
        # すべての整理済みリストをまとめて1つのデータフレームに追加する
        combined_results = []
        for file_path, integers in results.items():
            processed_lists = process_judge_2_ect(integers)
            combined_results.extend(processed_lists)
        
        # データフレームに変換してExcelシートに書き出し（行と列を入れ替えて書き出す）
        df = pd.DataFrame(combined_results)
        df.to_excel(writer, sheet_name=folder_name, header=False, index=False)

def main():
    root_folder_path = select_folder()
    if not root_folder_path:
        print("フォルダが選択されませんでした。")
        return

    # 選択されたフォルダ内のサブフォルダを全て取得
    for folder_name in os.listdir(root_folder_path):
        folder_path = os.path.join(root_folder_path, folder_name)
        if os.path.isdir(folder_path):
            print(f"{folder_name} の処理を開始します...")

            # JUDGE_20.ECTを優先的に検索
            judge_20_files = find_files(folder_path, 'JUDGE_20.ECT')
            if judge_20_files:
                print(f"{folder_name} に JUDGE_20.ECT ファイルが見つかりました。JUDGE_20.ECT を処理します。")
                files = judge_20_files
            else:
                # JUDGE_20.ECT がない場合は JUDGE_2.ECT を検索
                judge_2_files = find_files(folder_path, 'JUDGE_2.ECT')
                if judge_2_files:
                    print(f"{folder_name} に JUDGE_2.ECT ファイルが見つかりました。JUDGE_2.ECT を処理します。")
                    files = judge_2_files
                else:
                    print(f"{folder_name} に JUDGE_2.ECT ファイルが見つかりませんでした。")
                    continue

            all_integers = []  # 全ての JUDGE ファイルのデータを格納するリスト
            for file_path in files:
                integers = read_ect_file(file_path)
                all_integers.extend(integers)  # 全てのデータを結合

            results = {'combined': all_integers}  # 結合したデータを results に格納
            write_to_excel(results, folder_name)  # Excel に書き込み
            print(f"{folder_name} の結果が {folder_name}.xlsx に書き出されました。")

if __name__ == "__main__":
    main()

2011 の処理を開始します...
2011 に JUDGE_2.ECT ファイルが見つかりました。JUDGE_2.ECT を処理します。
2011 の結果が 2011.xlsx に書き出されました。
